# Data Modeling with Apache Cassandra
#### Udacity Data Engineering Nanodegree | Project 2 | Data Modeling with Cassandra | Manny Brar


#### Import Python packages 

In [33]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [34]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [35]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [36]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [37]:

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.01'])
    # For connection to locally installed Apache Cassandra
    session = cluster.connect()
    # To establish connection and begin executing queries, need a session
except Exception as e:
    print(e)


#### Create Keyspace

In [38]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION=
    {'class': 'SimpleStrategy', 'replication_factor' : 1}""")
    
except Exception as e:
    print(e)


#### Set Keyspace

In [39]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)



## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Q1

In [40]:
# CREATE TABLES
query1="CREATE TABLE IF NOT EXISTS songplay_"
query1= query1 + "(sessionId int, itemInSession int, artist_name text, song text, length decimal,  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)
    """ ^^^^ Here we create a new table if it does not currently exist and we assign it as songplay_, 
    we then specify our columns and data types for each column as well as setting our 
    PRIMARY KEY and Partition key ^^^"""

In [41]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO songplay_ (sessionId, itemInSession, artist_name, song, length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
    """ ^^^^ Next we read in the CSV file and Insert the data into the songplay_ table we created ^^^"""

In [42]:
query1="SELECT artist_name, length, song FROM songplay_ WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows=session.execute(query1)
except Exception as e:
    print(e)

    
print('Artist Name','|', 'Length', '|', 'Song Title')
for row in rows:
    print(row.artist_name,'|', row.length,'|', row.song)

    """ ^^^^ Here we execute our SELECT statement to answer question 1 and we print the results below ^^^"""                   

Artist Name | Length | Song Title
Faithless | 495.3073 | Music Matters (Mark Knight Dub)


## Conclusion Q1

### Q1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


For the purposes of this project we are modeling our data using a NoSQL schema utilizing Apache Cassandra and to accomplish this we first need to consider the questions we are asking our data. We must build our tables in accordance with the queries we intend to run in order to answer our questions. The intention here is to return the name of the artist, song title, and length of the song, in relation with the provided sesionId and itemInSession values.

Next we construct our SELECT statement and it should look something like this:
SELECT artist name, song title, song length (<<< Here we identify the values we would like returned)

FROM songplay_ (<<< Here we specify from which table we are requesting the results from)

WHERE sessionId = value_X AND itemInSession = value_Y (<<< Next we specify the parameters of the values returned)

Now, we can create our table using 'CREATE TABLE' and 'IF NOT EXISTS' to make sure that we only create the table if it currenlty does not exist. Next we need to select the the PRIMARY KEY. The PRIMARY KEY is required to be a unique identifier for each row in the table and because our query is based on sessionId and itemInSession, we will need to use both together as a unique indentifier.


# Q2

In [43]:
query2 = "CREATE TABLE IF NOT EXISTS songplay_user"
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist_name text, song text, first_name text, last_name text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
    
    """ ^^^^ Here we create a new table if it does not currently exist and we assign it as songplay_user, 
    we then specify our columns and data types for each column as well as setting our 
    PRIMARY KEY and Partition key ^^^"""
    
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader2 = csv.reader(f)
    next(csvreader2) # skip header
    for line in csvreader2:
        query2 = "INSERT INTO songplay_user (userId, sessionId, itemInSession, artist_name, song, first_name, last_name)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
    
    """ ^^^^ Next we read in the CSV file and Insert the data into the songplay_user table we created ^^^"""
        
        
s2 = "SELECT artist_name, song, first_name, last_name FROM songplay_user WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(s2)
except Exception as e:
    print(e)
print('Artist Name', '|', 'Song Title', '|', 'User First Name', '|', 'User Last Name')
for row in rows:
    print (row.artist_name,'|', row.song,'|', row.first_name,'|', row.last_name)
    
    """ ^^^^ Here we execute our SELECT statement to answer question 2 and we print the results below ^^^"""                   

Artist Name | Song Title | User First Name | User Last Name
Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


## Conclusion Q2

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Again we must build our tables in accordance with the queries we intend to run in order to answer our questions. 
The intention here is to return the name of the artist, song title (sorted by itemInSession), user first name and user last name in relation with the provided userId and sesionId values.

Next we construct our SELECT statement and it should look something like this:
SELECT artist name, song title, user first name, user last name (<<< Here we identify the values we would like returned)

FROM songplay_user (<<< Here we specify from which table we are requesting the results from)

WHERE userId = value_X AND sessionId = value_Y (<<< Next we specify the parameters of the values returned)

Now, we can create our table using 'CREATE TABLE' and 'IF NOT EXISTS' to make sure that we only create the table if it currenlty does not exist. Next we need to select the the PRIMARY KEY. The PRIMARY KEY is required to be a unique identifier for each row in the table and because our query is based on userId and sessionId, we will need to use both together as our partition key and since we will need to sort by itemInSession, we add that as our clustering column.

# Q3

In [44]:
query3 = "CREATE TABLE IF NOT EXISTS user_song_hist"
query3 = query3 + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

    """ ^^^^ Here we create a new table if it does not currently exist and we assign it as user_song_hist, 
    we then specify our columns and data types for each column as well as setting our 
    PRIMARY KEY and Partition key ^^^"""
    
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader3 = csv.reader(f)
    next(csvreader3) # skip header
    for line in csvreader3:
        query3 = "INSERT INTO user_song_hist (song, user_id, first_name, last_name)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

    """ ^^^^ Next we read in the CSV file and Insert the data into the user_song_hist table we created ^^^"""

        
        
        
s3 = """SELECT first_name, last_name
           FROM user_song_hist
           WHERE song = 'All Hands Against His Own'"""

try:
    rows = session.execute(s3)
except Exception as e:
    print(e)
print('First Name', '|', 'Last Name')
for row in rows:
    print (row.first_name, '|', row.last_name)
    
    
    """ ^^^^ Here we execute our SELECT statement to answer question 3 and we print the results below ^^^"""                   

First Name | Last Name
Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


## Conclusion Q3

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The intention here is to return the first and last name of the users who listened to a certain song.

To start we construct our SELECT statement and it should look something like this:
SELECT user first name, user last name (<<< Here we identify the values we would like returned)

FROM user_song_hist (<<< Here we specify from which table we are requesting the results from)

WHERE song = value_X (<<< Next we specify the parameters of the values returned)

Now, we can create our table using 'CREATE TABLE' and 'IF NOT EXISTS' to make sure that we only create the table if it currenlty does not exist. Next we need to select the the PRIMARY KEY. The PRIMARY KEY is required to be a unique identifier for each row in the table and because our query is based on users that listened to a certain song, we will need to use both together as our PRIMARY KEY.

### Drop the tables before closing out the sessions

In [45]:
session.execute("""DROP TABLE songplay_""")

session.execute("""DROP TABLE songplay_user""")

session.execute("""DROP TABLE user_song_hist""")

# Here we can drop all our created tables

### Close the session and cluster connection¶

In [46]:
session.shutdown()
cluster.shutdown()